cargo los datos del drive

In [1]:
from google.colab import drive
drive.mount('/content/drive');

Mounted at /content/drive


In [2]:
# importamos librerias para el manejo de datos pandas para manejo de datos y numpy operaciones con numeros
import numpy as np
import pandas as pd

#import tools

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [32]:
#cargo los datos
train_1=pd.read_csv('/content/drive/MyDrive/bigdata3/proyecto homicidios/columnas_necesarias.csv')

In [33]:
train_1.head()

,Fecha,Departamento,Dia,HoraEstandarizada,Zona,Modalidad,Movil_Agresor,Movil_Victima,Edad,Sexo
0,08/12/2010 12:00:00 AM,VALLE,Miercoles,1,URBANA,ARMA DE FUEGO,A PIE,A PIE,18,MASCULINO
1,01/01/2010 12:00:00 AM,ANTIOQUIA,Viernes,8,URBANA,ARMA DE FUEGO,A PIE,A PIE,32,MASCULINO
2,08/11/2010 12:00:00 AM,NORTE DE SANTANDER,Lunes,7,URBANA,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,A PIE,48,MASCULINO
3,2010-01-01 00:00:00,CAUCA,Lunes,23,URBANA,ARMA DE FUEGO,A PIE,A PIE,26,MASCULINO
4,2010-01-01 00:00:00,CUNDINAMARCA,Domingo,0,URBANA,ARMA BLANCA,A PIE,A PIE,43,MASCULINO


In [ ]:
train_1_ = train[['Fecha', 'Departamento', 'Dia', 'HoraEstandarizada', 'Zona', 'Modalidad', 'Movil_Agresor', 'Movil_Victima', 'Edad', 'Sexo']]
train_1_.head()

In [34]:
#para la columna fecha
#convierto la columna Fecha a tipo fecha con to_datetime()
df = pd.to_datetime(train_1['Fecha'])
#cargo el train_1 en un dataframe para agregar la columna mes
train_1 = pd.DataFrame(train_1)
#agrego la columna mes al train_1
train_1['Mes'] = df.dt.month

#hacemos lo mismo para la columna Departamento
train_1=pd.concat([train_1,pd.get_dummies(train_1['Departamento'],prefix='Departamento_')],axis=1)

#hacemos lo mismo para la columna dia
train_1=pd.concat([train_1,pd.get_dummies(train_1['Dia'],prefix='Dia_')],axis=1)

#luego de cambiar el texto por numeros en el dia, normalizo (valor - min) / (max - min)
#df_norm = (train_1['Dia'] - train_1['Dia'].min()) / (train_1['Dia'].max()-train_1['Dia'].min())
#ya habia agregado el train_1 a un data frame en fecha ahora agrego la columna dia normalizada al train_1
#train_1['dia_normalizado'] = df_norm

#para la columna HoraEstandarizada que se sacó solo la hora mas rápido con excel
train_1=pd.concat([train_1,pd.get_dummies(train_1['HoraEstandarizada'],prefix='Hora_')],axis=1)

#dummies crea categorias a paetir de los valores existentes para Zona
train_1=pd.concat([train_1,pd.get_dummies(train_1['Zona'],prefix='Zona_')],axis=1)

#hacemos lo mismo para la columna Modalidad
train_1=pd.concat([train_1,pd.get_dummies(train_1['Modalidad'],prefix='Modalidad_')],axis=1)

#hacemos lo mismo para la columna Movil_Agresor
train_1=pd.concat([train_1,pd.get_dummies(train_1['Movil_Agresor'],prefix='Movil_Agresor_')],axis=1)

#hacemos lo mismo para la columna Movil_Victima
train_1=pd.concat([train_1,pd.get_dummies(train_1['Movil_Victima'],prefix='Movil_Victima_')],axis=1)

#normalizo la columna edad (valor - min) / (max - min)
df_norm2 = (train_1['Edad'] - train_1['Edad'].min()) / (train_1['Edad'].max()-train_1['Edad'].min())
#ya habia agregado el train_1 a un data frame en fecha ahora agrego la columna edad normalizada al train_1
train_1['edad_normalizada'] = df_norm2

#hacemos lo mismo para la columna sexo
train_1=pd.concat([train_1,pd.get_dummies(train_1['Sexo'],prefix='Sexo_')],axis=1)

train_1.head()

,Fecha,Departamento,Dia,HoraEstandarizada,Zona,Modalidad,Movil_Agresor,Movil_Victima,Edad,Sexo,Mes,Departamento__AMAZONAS,Departamento__ANTIOQUIA,Departamento__ARAUCA,Departamento__ATLANTICO,Departamento__BOLIVAR,Departamento__BOYACA,Departamento__CALDAS,Departamento__CAQUETA,Departamento__CASANARE,Departamento__CAUCA,Departamento__CESAR,Departamento__CHOCO,Departamento__CORDOBA,Departamento__CUNDINAMARCA,Departamento__GUAINIA,Departamento__GUAJIRA,Departamento__GUAVIARE,Departamento__HUILA,Departamento__MAGDALENA,Departamento__META,Departamento__NARINO,Departamento__NORTE DE SANTANDER,Departamento__PUTUMAYO,Departamento__QUINDIO,Departamento__RISARALDA,Departamento__SAN ANDRES,Departamento__SANTANDER,Departamento__SUCRE,Departamento__TOLIMA,...,Modalidad__MOTO BOMBA,Modalidad__OLLA BOMBA,Modalidad__PAQUETE BOMBA,Modalidad__PERSONA BOMBA,Modalidad__POLVORA(FUEGOS PIROTECNICOS),Modalidad__PRENDAS DE VESTIR,Modalidad__PUNZANTES,Modalidad__QUIMICOS,Modalidad__ROCKET,Modalidad__SIN EMPLEO DE ARMAS,Modalidad__SUSTANCIAS TOXICAS,Modalidad__VENENO,Movil_Agresor__A PIE,Movil_Agresor__BICICLETA,Movil_Agresor__CONDUCTOR BUS,Movil_Agresor__CONDUCTOR MOTOCICLETA,Movil_Agresor__CONDUCTOR TAXI,Movil_Agresor__CONDUCTOR VEHICULO,Movil_Agresor__PASAJERO AERONAVE,Movil_Agresor__PASAJERO BARCO,Movil_Agresor__PASAJERO BUS,Movil_Agresor__PASAJERO MOTOCICLETA,Movil_Agresor__PASAJERO TAXI,Movil_Agresor__PASAJERO VEHICULO,Movil_Agresor__VEHICULO,Movil_Victima__A PIE,Movil_Victima__BICICLETA,Movil_Victima__CONDUCTOR BUS,Movil_Victima__CONDUCTOR MOTOCICLETA,Movil_Victima__CONDUCTOR TAXI,Movil_Victima__CONDUCTOR VEHICULO,Movil_Victima__PASAJERO BARCO,Movil_Victima__PASAJERO BUS,Movil_Victima__PASAJERO MOTOCICLETA,Movil_Victima__PASAJERO TAXI,Movil_Victima__PASAJERO VEHICULO,Movil_Victima__VEHICULO,edad_normalizada,Sexo__FEMENINO,Sexo__MASCULINO
0,08/12/2010 12:00:00 AM,VALLE,Miercoles,1,URBANA,ARMA DE FUEGO,A PIE,A PIE,18,MASCULINO,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.18,0,1
1,01/01/2010 12:00:00 AM,ANTIOQUIA,Viernes,8,URBANA,ARMA DE FUEGO,A PIE,A PIE,32,MASCULINO,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.32,0,1
2,08/11/2010 12:00:00 AM,NORTE DE SANTANDER,Lunes,7,URBANA,ARMA DE FUEGO,CONDUCTOR MOTOCICLETA,A PIE,48,MASCULINO,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.48,0,1
3,2010-01-01 00:00:00,CAUCA,Lunes,23,URBANA,ARMA DE FUEGO,A PIE,A PIE,26,MASCULINO,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.26,0,1
4,2010-01-01 00:00:00,CUNDINAMARCA,Domingo,0,URBANA,ARMA BLANCA,A PIE,A PIE,43,MASCULINO,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.43,0,1


In [35]:
#borro las columnas que categorice con drop, el implace me dice que ejecuta todo sobre la misma 
# variable, es equivalente a train_1=train_1
train_1.drop(labels=['Fecha'],axis=1,inplace=True)
train_1.drop(labels=['Departamento'],axis=1,inplace=True)
train_1.drop(labels=['Dia'],axis=1,inplace=True)
train_1.drop(labels=['HoraEstandarizada'],axis=1,inplace=True)
train_1.drop(labels=['Zona'],axis=1,inplace=True)
train_1.drop(labels=['Modalidad'],axis=1,inplace=True)
train_1.drop(labels=['Movil_Agresor'],axis=1,inplace=True)
train_1.drop(labels=['Movil_Victima'],axis=1,inplace=True)
train_1.drop(labels=['Edad'],axis=1,inplace=True)
train_1.drop(labels=['Sexo'],axis=1,inplace=True)
#train_1.drop(labels=['Mes'],axis=1,inplace=True)
train_1.head()


,Mes,Departamento__AMAZONAS,Departamento__ANTIOQUIA,Departamento__ARAUCA,Departamento__ATLANTICO,Departamento__BOLIVAR,Departamento__BOYACA,Departamento__CALDAS,Departamento__CAQUETA,Departamento__CASANARE,Departamento__CAUCA,Departamento__CESAR,Departamento__CHOCO,Departamento__CORDOBA,Departamento__CUNDINAMARCA,Departamento__GUAINIA,Departamento__GUAJIRA,Departamento__GUAVIARE,Departamento__HUILA,Departamento__MAGDALENA,Departamento__META,Departamento__NARINO,Departamento__NORTE DE SANTANDER,Departamento__PUTUMAYO,Departamento__QUINDIO,Departamento__RISARALDA,Departamento__SAN ANDRES,Departamento__SANTANDER,Departamento__SUCRE,Departamento__TOLIMA,Departamento__VALLE,Departamento__VAUPES,Departamento__VICHADA,Dia__Domingo,Dia__Jueves,Dia__Lunes,Dia__Martes,Dia__Miercoles,Dia__Sabado,Dia__Viernes,...,Modalidad__MOTO BOMBA,Modalidad__OLLA BOMBA,Modalidad__PAQUETE BOMBA,Modalidad__PERSONA BOMBA,Modalidad__POLVORA(FUEGOS PIROTECNICOS),Modalidad__PRENDAS DE VESTIR,Modalidad__PUNZANTES,Modalidad__QUIMICOS,Modalidad__ROCKET,Modalidad__SIN EMPLEO DE ARMAS,Modalidad__SUSTANCIAS TOXICAS,Modalidad__VENENO,Movil_Agresor__A PIE,Movil_Agresor__BICICLETA,Movil_Agresor__CONDUCTOR BUS,Movil_Agresor__CONDUCTOR MOTOCICLETA,Movil_Agresor__CONDUCTOR TAXI,Movil_Agresor__CONDUCTOR VEHICULO,Movil_Agresor__PASAJERO AERONAVE,Movil_Agresor__PASAJERO BARCO,Movil_Agresor__PASAJERO BUS,Movil_Agresor__PASAJERO MOTOCICLETA,Movil_Agresor__PASAJERO TAXI,Movil_Agresor__PASAJERO VEHICULO,Movil_Agresor__VEHICULO,Movil_Victima__A PIE,Movil_Victima__BICICLETA,Movil_Victima__CONDUCTOR BUS,Movil_Victima__CONDUCTOR MOTOCICLETA,Movil_Victima__CONDUCTOR TAXI,Movil_Victima__CONDUCTOR VEHICULO,Movil_Victima__PASAJERO BARCO,Movil_Victima__PASAJERO BUS,Movil_Victima__PASAJERO MOTOCICLETA,Movil_Victima__PASAJERO TAXI,Movil_Victima__PASAJERO VEHICULO,Movil_Victima__VEHICULO,edad_normalizada,Sexo__FEMENINO,Sexo__MASCULINO
0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.18,0,1
1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.32,0,1
2,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.48,0,1
3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.26,0,1
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.43,0,1


In [36]:
# Y = mX + b  :: Y = wX + b
y = train_1['Mes']
print(y)

0          8
1          1
2          8
3          1
4          1
          ..
102300    11
102301    11
102302    11
102303    11
102304    11
Name: Mes, Length: 102305, dtype: int64


In [37]:
x = train_1
x.drop('Mes', axis=1, inplace=True)
print(x)

        Departamento__AMAZONAS  ...  Sexo__MASCULINO
0                            0  ...                1
1                            0  ...                1
2                            0  ...                1
3                            0  ...                1
4                            0  ...                1
...                        ...  ...              ...
102300                       0  ...                1
102301                       0  ...                1
102302                       0  ...                0
102303                       0  ...                1
102304                       0  ...                1

[102305 rows x 130 columns]


In [38]:
# Separar los datos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.176)

In [24]:
print('== Algunos clasificadores ==')
#print('Regresión logística sin movilidad')
logReg = KNeighborsRegressor()
logReg.fit(x_train, y_train)
print('Exactitud KNR (train)', logReg.score(x_train, y_train))
print('Exactitud KNR (test)', logReg.score(x_test, y_test))

== Algunos clasificadores ==
Exactitud KNR (train) 0.2194655969906677
Exactitud KNR (test) -0.1565982193707398


In [39]:
print('== Algunos clasificadores ==')
#print('Regresión logística sin movilidad')
logReg = LogisticRegression()
logReg.fit(x_train, y_train)
print('Exactitud LogisticRegresion (train)', logReg.score(x_train, y_train))
print('Exactitud LogisticRegresion (test)', logReg.score(x_test, y_test))

== Algunos clasificadores ==
Exactitud LogisticRegresion (train) 0.1197141820414771
Exactitud LogisticRegresion (test) 0.10901863677831357


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [23]:
#ahora con ('Arbol de Decisión sin Movilidad')
treeR= DecisionTreeRegressor()
treeR.fit(x_train, y_train)
#y_predict = treeR.predict(x_val)
print('Precisión DecisionTreeRegressor (train)', treeR.score(x_train, y_train))
print('Precisión DecisionTreeRegressor (test)', treeR.score(x_test, y_test))

Precisión DecisionTreeRegressor (train) 0.8102408822402382
Precisión DecisionTreeRegressor (test) -0.7880989934107817


In [31]:
#ahora con ('Arbol de Decisión')
treeR= DecisionTreeRegressor()
treeR.fit(x_train, y_train)
#y_predict = treeR.predict(x_val)
print('Precisión DecisionTreeRegressor (train)', treeR.score(x_train, y_train))
print('Precisión DecisionTreeRegressor (test)', treeR.score(x_test, y_test))

Precisión DecisionTreeRegressor (train) 0.8674589182656491
Precisión DecisionTreeRegressor (test) -0.8247405711804485


In [13]:
print('== Algunos clasificadores ==')
#print('Regresión logística')
logReg = LogisticRegression()
logReg.fit(x_train, y_train)
print('Exactitud LogisticRegression (train)', logReg.score(x_train, y_train))
print('Exactitud LogisticRegression (test)', logReg.score(x_test, y_test))

== Algunos clasificadores ==
Exactitud LogisticRegression (train) 0.11930945934630306
Exactitud LogisticRegression (test) 0.10706372996220513


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
y_prediclore = logReg.predict(x_val)
#grafica para el modelo logReg
import matplotlib.pyplot as plt
plt.plot(y_prediclore, label='Predicted')
plt.plot(y_test, label='Expected')
plt.title("mes LogisticRegression")
plt.legend()
plt.show()

In [ ]:
#guardo el modelo
import pickle
rlfile = open("/content/drive/MyDrive/bigdata3/modelos/modelo_mesLoRe.pickle",'wb')
pickle.dump(logReg,rlfile)
rlfile.close()

In [ ]:
y_predict = logReg.predict(x_val)
import matplotlib.pyplot as plt
#plt.plot(y_predictknn, label='Predicted')
plt.plot(y_test, label='Expected')
plt.title("Zona")
plt.legend()
plt.show()

In [ ]:
#guardo el modelo
import pickle
rlfile = open("/content/drive/MyDrive/bigdata3/modelos/modelo_mesTreeR.pickle",'wb')
pickle.dump(logReg,rlfile)
rlfile.close()

In [ ]:
y_predicTreeRt = treeR.predict(x_val)
import matplotlib.pyplot as plt
plt.plot(y_predicTreeRt, label='Predicted')
plt.plot(y_test, label='Expected')
plt.title("Mes DecisionTreeRegressor")
plt.legend()
plt.show()